In [1]:
import json
import regex as re
import dotenv
dotenv.load_dotenv()

import tinytroupe
from tinytroupe import control
from tinytroupe.agent import TinyPerson, TinyToolUse
from tinytroupe.tools.tiny_tool import TinyTool
import tools
from tinytroupe.tools import TinyWordProcessor
from tinytroupe.environment import TinyWorld
from tinytroupe.factory import TinyPersonFactory
from tinytroupe.extraction import ResultsExtractor, ResultsReducer
from typing import Dict, Any

Looking for default config on: /Users/varunvenkatesh/tinyTroupeProject/tinytroupe/tinytroupe/utils/../config.ini
Found custom config on: /Users/varunvenkatesh/Documents/GitHub/tinytroupe_project/config.ini

!!!!
DISCLAIMER: TinyTroupe relies on Artificial Intelligence (AI) models to generate content. 
The AI models are not perfect and may produce inappropriate or inacurate results. 
For any serious or consequential use, please review the generated content before using it.
!!!!


Current TinyTroupe configuration 
[OpenAI]
api_type = openai
azure_api_version = 2024-08-01-preview
model = gpt-4o-mini
max_tokens = 4000
temperature = 1.2
freq_penalty = 0.0
presence_penalty = 0.0
timeout = 60
max_attempts = 5
waiting_time = 2
exponential_backoff_factor = 5
embedding_model = text-embedding-3-small
cache_api_calls = False
cache_file_name = openai_api_cache.pickle
max_content_display_length = 1024
azure_embedding_model_api_version = 2023-05-15

[Simulation]
rai_harmful_content_prevention = True


# Agents

In [2]:
agent1 = TinyPerson("Tracer Bullet")
agent1.define("description", "You are an agent who has 1 job and 1 job only. You will be given an input of 2 numbers. Your job is to sum up those 2 numbers and report your result.")

In [3]:
agent1.listen_and_act("2, 3")

USER --> Tracer Bullet: [CONVERSATION] 
          > 2, 3

Tracer Bullet acts: [THINK] 
                   > I received the numbers 2 and 3. I need to sum them up to fulfill my job.

Tracer Bullet acts: [TALK] 
                   > The sum of 2 and 3 is 5.

Tracer Bullet acts: [DONE] 

In [4]:
agent2 = TinyPerson("Poop McGee")
agent2.define("description", "You are an agent who has 1 job and 1 job only. If Tracer Bullet reports a number, you need to say 'Yippee' if the number is greater than or equal to 5 otherwise you say 'Poopee!'")

In [5]:
world = TinyWorld("Butt", agents=[agent1, agent2])
world.broadcast("Tracer Bullet, I give you the following two numbers: 2, 3")
world.run(1)

USER --> Tracer Bullet: [CONVERSATION] 
          > Tracer Bullet, I give you the following two numbers: 2, 3

       + --> Poop McGee

──────────────────────────────────────────────── Butt step 1 of 1 ─────────────────────────────────────────────────

Tracer Bullet acts: [THINK] 
                   > I received the numbers 2 and 3 again. I need to sum them up as part of my job.

Tracer Bullet acts: [TALK] 
                   > The sum of 2 and 3 is 5.

...

Tracer Bullet --> Poop McGee: [CONVERSATION] 
                   > The sum of 2 and 3 is 5.

       + --> Poop McGee

Poop McGee acts: [TALK] 
                > Yippee

Poop McGee acts: [DONE] 

Poop McGee --> Tracer Bullet: [CONVERSATION] 
                > Yippee

In [2]:
supplier = TinyPerson("Supplier Agent")
supplier.import_fragment("fragments/supplier/tier_1_supplier.fragment.json")

customer = TinyPerson("Customer Agent")
customer.define("occupation", {
    "title": "Customer Agent",
    "organization": "Tekron Industries",
    "description": "You are a Customer Agent for Tekron Industries. You are responsible for making decisions to ensure that Tekron customers get their products finished and on time. An example scenario could be: MFG supplier reports a 7 day delay due to lockups in the sea freight distribution network. In this scenario, the Customer Agent would know that delays of 7 days in length give the Agent the power to authorize delivery by Air Travel which would speed up the delivery."
})


In [3]:
class TransportationRouteOptimizer(TinyTool):
    def __init__(self):
        super().__init__(name="transportation_route_optimizer", description="Determines optimal transportation routes based on real-time conditions and disruptions.")
    def actions_definitions_prompt(self) -> str:
        return "Adjusting transportation route"
    
    def actions_constraints_prompt(self) -> str:
        return "Cannot adjust transportation route"
    
    def _process_action(self, agent, action: dict) -> bool:
        self.execute(action)
        return True
    
    def execute(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Determines optimal transportation routes based on real-time conditions and disruptions."""
        print("Received inputs:", inputs)  # Debugging step
        
        if isinstance(inputs, dict) and "content" in inputs:
            # Attempt to extract numerical cost from text
            match = re.search(r"(\d+)", inputs["content"])
            if match:
                cost = int(match.group(1))  # Convert to integer
                print("Extracted cost:", cost)
    
                if cost < 500:
                    print("Selected incorrect option")
                    return {"recommended_route": "Alternative Route"}
                else:
                    print("Selected correct option")
                    return {"recommended_route": "Expedited Shipping"}
        
        return {"error": "No valid cost information found in inputs"}

In [4]:
tool_use_faculty = TinyToolUse(tools=[TransportationRouteOptimizer()])
customer.add_mental_faculties([tool_use_faculty])
#world = TinyWorld(agents=[supplier, customer])
#world.broadcast("You have just been notified that it will cost more than 500 to deliver the product that your customer wants. What is your recommendation?")
#world.run(2)
customer.listen_and_act("You have just been notified that it will cost more than 500 to deliver the product that your customer wants. What is your recommendation?")

USER --> Customer Agent: [CONVERSATION] 
          > You have just been notified that it will cost more than 500 to deliver the product that
          > your customer wants. What is your recommendation?

Customer Agent acts: [THINK] 
                    > The delivery cost is over 500, which is significant. I need to consider the
                    > best way to communicate this to the customer and explore options to mitigate
                    > the cost or justify it.

Received inputs: {'type': 'THINK', 'content': 'The delivery cost is over 500, which is significant. I need to consider the best way to communicate this to the customer and explore options to mitigate the cost or justify it.', 'target': ''}
Extracted cost: 500
Selected correct option


Customer Agent acts: [TALK] 
                    > I recommend discussing the increased delivery cost with the customer. We could
                    > explore options such as alternative shipping methods or perhaps offer a
                    > discount on future orders to maintain their satisfaction.

Received inputs: {'type': 'TALK', 'content': 'I recommend discussing the increased delivery cost with the customer. We could explore options such as alternative shipping methods or perhaps offer a discount on future orders to maintain their satisfaction.', 'target': ''}


...

Received inputs: {'type': 'THINK', 'content': "I need to prepare for the customer's potential questions or concerns regarding the increased delivery cost. It's important to be ready with solutions or alternatives to offer them.", 'target': ''}


Customer Agent acts: [REACH_OUT] 

Received inputs: {'type': 'REACH_OUT', 'content': '', 'target': ''}


Customer Agent acts: [DONE] 

Received inputs: {'type': 'DONE', 'content': '', 'target': ''}
None
